In [1]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

In [2]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    
# grab the dimensions of the frame and then construct a blob from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))

# pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)

# initialize our list of faces, their corresponding locations and the list of predictions from our face mask network
    faces = []
    locs = []
    preds = []

# loop over the detections
    for i in range(0, detections.shape[2]):
# extract the confidence (i.e., probability) associated with the detection
        confidence = detections[0, 0, i, 2]

# filter out weak detections by ensuring the confidence is greater than the minimum confidence
        if confidence > 0.5:
# compute the (x, y)-coordinates of the bounding box for the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

# ensure the bounding boxes fall within the dimensions of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

# extract the face ROI, convert it from BGR to RGB channel ordering, resize it to 224x224, and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

# add the face and bounding boxes to their respective lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))

# only make a predictions if at least one face was detected
    if len(faces) > 0:
# for faster inference we'll make batch predictions on *all* aces at the same time rather than one-by-one predictions
# in the above `for` loop
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

# return a 2-tuple of the face locations and their corresponding locations
    return (locs, preds)

In [3]:
# load our serialized face detector model from disk
prototxtPath=r"D:\AI Course\Deep Learning\Face Mask D\face_detector\deploy.prototxt"
weightsPath=r"D:\AI Course\Deep Learning\Face Mask D\face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet=cv2.dnn.readNet(prototxtPath,weightsPath)

# load the face mask detector model from disk
maskNet = load_model(r"D:\AI Course\Deep Learning\Face Mask D\Face_mask_detector.h5")

# initialize the video stream
print("[INFO] starting video stream...")
vs=VideoStream(src=0).start()
time.sleep(2.0)

[INFO] starting video stream...


In [4]:
# loop over the frames from the video stream
while True:
#grab the frame from the threaded video stream and resize it to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

#detect faces in the frame and determine if they are wearing a face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

# loop over the detected face locations and their corresponding locations
    for (box, pred) in zip(locs, preds):
        
# unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

# determine the class label and color we'll use to draw the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

# include the probability in the label
        #label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        
# display the label and bounding box rectangle on the outputframe
        if(label=="Mask"):
            
            cv2.putText(frame,"Mask: You are allowed", (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        elif(label=="No Mask"):
            lab="No Mask: You are not allowed"
            cv2.putText(frame, lab, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

# show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

# if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()

vs.stop()

(1, 1, 200, 7)
1/1 [==============================] - 2s 2s/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 65ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 57ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 70ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 63ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 89ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 58ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 49ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 59ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 58ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 48ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 54ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 53ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 45ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/st

(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200

(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
1/1 [==============================] - 0s 49ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 37ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 39ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 40ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 38ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 39ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 38ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 35ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 36ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 39ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 200, 7)
1/1 [===

1/1 [==============================] - 0s 50ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 45ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 43ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 44ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 42ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 40ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 41ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 48ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 46ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 38ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 46ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 48ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 45ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 49ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 45ms/step
(1, 1, 200

1/1 [==============================] - 0s 54ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 58ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 47ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 51ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 48ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 53ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 57ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 55ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 48ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 46ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 49ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 57ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 63ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 56ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 52ms/step
(1, 1, 200

...

(1, 1, 200, 7)
1/1 [==============================] - 0s 113ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 121ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 93ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 143ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 134ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 215ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 208ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 96ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 98ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 97ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 98ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 81ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 78ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 109ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0

1/1 [==============================] - 0s 94ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 81ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 95ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 98ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 111ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 95ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 89ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 91ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 89ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 71ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 109ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 86ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 94ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 70ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 75ms/step
(1, 1, 2

1/1 [==============================] - 0s 84ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 80ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 82ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 74ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 82ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 89ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 94ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 92ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 74ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 94ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 93ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 85ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 87ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 94ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 85ms/step
(1, 1, 200

(1, 1, 200, 7)
1/1 [==============================] - 0s 91ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 97ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 112ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 124ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 101ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 98ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 141ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 180ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 120ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 114ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 103ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 130ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 113ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 101ms/step
(1, 1, 200, 7)
1/1 [==============================]

1/1 [==============================] - 0s 97ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 74ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 106ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 88ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 108ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 94ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 93ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 95ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 128ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 148ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 238ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 332ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 165ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 126ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 113ms/step
(

NameError: name 'q' is not defined